<a href="https://colab.research.google.com/github/victor-roris/NLPlearning/blob/master/NER/Test_Rasa_1_10_custom_data_(Fail).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train RASA_NLU with custom data

In this notebook I test to train a RASA_NLU model with my own training data. But it always fails by Out of memory (OOM) error. 

In [5]:
from IPython.display import clear_output 

## Train rasa_nlu by console

[Blog](https://towardsdatascience.com/a-beginners-guide-to-rasa-nlu-for-intent-classification-and-named-entity-recognition-a4f0f76b2a96)

* Setup and installation

In [6]:
!pip install rasa
clear_output()

In [7]:
!pip install -U ipython
clear_output()

In [24]:
import rasa
rasa.__version__

'1.10.12'

* Activate the virtual environment

In [8]:
! mkdir rasa
%cd rasa
! rasa init --no-prompt

/content/rasa
Welcome to Rasa! 🤖

To get started quickly, an initial project will be created.
If you need some help, check out the documentation at https://rasa.com/docs/rasa.

Created project directory at '/content/rasa'.
Finished creating project structure.
Training an initial model...
Training Core model...
2020-09-03 11:33:37 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
2020-09-03 11:33:38 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
Processed Story Blocks: 100% 5/5 [00:00<00:00, 3338.88it/s, # trackers=1]
Processed Story Blocks: 100% 5/5 [00:00<00:00, 1446.71it/s, # trackers=5]
Processed Story Blocks: 100% 5/5 [00:00<00:00, 368.10it/s, # trackers=20]
Processed Story Blocks: 100% 5/5 [00:00<00:00, 245.60it/s, # trackers=24]
Processed trackers: 100% 5/5 [00:00<00:00, 2093.80it/s, # actions=16]
Processed actions: 16it [00:00, 6398.02it/s, # examples=16]
Processed trackers: 100% 231/231 [00:00<0

* Training data

I copy my training data in the file `rasa/data/nlu.md` file.

* Train

In [9]:
!rasa train nlu

Training NLU model...
2020-09-03 11:42:47 INFO     rasa.nlu.training_data.training_data  - Training data stats:
2020-09-03 11:42:47 INFO     rasa.nlu.training_data.training_data  - Number of intent examples: 294 (3 distinct intents)
2020-09-03 11:42:47 INFO     rasa.nlu.training_data.training_data  -   Found intents: 'agreement_title', 'agreement_parties', 'other_agreement_text'
2020-09-03 11:42:47 INFO     rasa.nlu.training_data.training_data  - Number of response examples: 0 (0 distinct responses)
2020-09-03 11:42:47 INFO     rasa.nlu.training_data.training_data  - Number of entity examples: 218 (14 distinct entities)
2020-09-03 11:42:47 INFO     rasa.nlu.training_data.training_data  -   Found entity types: 'agreement_date_label', 'agreement_label', 'agreement_date', 'party2_label', 'agreement_type', 'party2_address', 'party2_regcountry', 'party1', 'party1_regcountry', 'party2_number', 'party1_label', 'party1_address', 'party1_number', 'party2'
2020-09-03 11:42:47 INFO     rasa.nlu.m

## Train rasa_nlu by python

In [18]:
%cd /content/

/content


* Install rasa (last version)

In [10]:
!pip install rasa
!pip install rasa[convert]
clear_output()

In [11]:
!pip install 'gast==0.2.2'
clear_output()

* Install spacy as pre-trained language model

In [12]:
!pip install spacy
!python -m spacy download en_core_web_md
!python -m spacy link en_core_web_md en --force
clear_output()

## Training data

I upload my training data in the root folder in a file called `nlu_nda.md`.

### Train a model

In [13]:
from rasa.nlu.model import Trainer
from rasa.nlu import config
from rasa.nlu.training_data import load_data
from rasa.nlu.test import run_evaluation

#### Interpreter 1: based in ConveRT

In [14]:
convertRT_pipeline = """
language: "en"

pipeline:
  - name: ConveRTTokenizer
  - name: ConveRTFeaturizer
  - name: RegexFeaturizer
  - name: LexicalSyntacticFeaturizer
  - name: CountVectorsFeaturizer
  - name: CountVectorsFeaturizer
    analyzer: "char_wb"
    min_ngram: 1
    max_ngram: 4
  - name: DIETClassifier
    epochs: 100
  - name: EntitySynonymMapper
"""

convertRT_pipeline_filepath = "convertRT_pipeline.yml"
f = open(convertRT_pipeline_filepath, "w")
f.write(convertRT_pipeline)
f.close()

In [15]:
configuration = config.load(convertRT_pipeline_filepath)

In [16]:
interpreter1 = Trainer(configuration)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [19]:
training_nda_data_filepath = "nlu_nda.md"
training_data = load_data(training_nda_data_filepath)
interpreter1.train(training_data)

Text batches: 100%|██████████| 5/5 [00:13<00:00,  2.71s/it]
Response batches: 0it [00:00, ?it/s]
/usr/local/lib/python3.6/dist-packages/rasa/utils/common.py:363: UserWarning: Misaligned entity annotation in message 'COOK SPACE Equioment Rental Agreement This Equipment Rental Agreement (“Agreement”) is effective as of the date of first payment (“Effective Date”), and is made between Mette Ltd, a Limited Company governed under the laws of England and Wales, with offices at 46 Hill Street, Belfast, Northern Ireland, BT1 2LB, and Glove Factory Studios, Brook Lane, Holt, Wiltshire BA14 6RL (“Owner”), and Strong Roots (Handy Food Innovation Ltd), with offices at The Root System, Fourth Floor, 25-26 Earlsfort Terrace, Dublin 2, Ireland and 2 Stephen Street, London, W1T 1AN, UK (“Renter”). Owner and Renter are hereinafter collectively referred to as “Parties”.' with intent 'agreement_parties'. Make sure the start and end values of entities in the training data match the token boundaries (e.g. 

ResourceExhaustedError: ignored

In [ ]:
interpreter1.persist

#### Interpreter 2: based in Spacy

In [20]:
spacy_pipeline = """
language: "en"

pipeline:
  - name: SpacyNLP
  - name: SpacyTokenizer
  - name: SpacyFeaturizer
  - name: RegexFeaturizer
  - name: LexicalSyntacticFeaturizer
  - name: CountVectorsFeaturizer
  - name: CountVectorsFeaturizer
    analyzer: "char_wb"
    min_ngram: 1
    max_ngram: 4
  - name: DIETClassifier
    epochs: 10
  - name: EntitySynonymMapper
"""

spacy_pipeline_filepath = "spacy_pipeline.yml"
f = open(spacy_pipeline_filepath, "w")
f.write(spacy_pipeline)
f.close()

In [21]:
configuration = config.load(spacy_pipeline_filepath)

In [22]:
interpreter2 = Trainer(configuration)

In [23]:
training_nda_data_filepath = "nlu_nda.md"
training_data = load_data(training_nda_data_filepath)
interpreter2.train(training_data)


Epochs:  10%|█         | 1/10 [00:08<01:17,  8.64s/it, t_loss=142.266, i_loss=6.551, entity_loss=133.962, i_acc=0.503, entity_f1=0.015]

ResourceExhaustedError: ignored